In [4]:
import importlib
import sys
sys.path.append('../src')

import qa
importlib.reload(qa)
import qa
from qa import QA

import pickle

In [2]:
# from datasets import load_dataset

# ds = load_dataset("TIGER-Lab/MMLU-Pro")

# mmlu = {}
# for ele in ds['test']:
#     if ele['category'] not in mmlu:
#         mmlu[ele['category']] = []
#     mmlu[ele['category']].append(ele)

# import random

# mmlu_samples = []
# for cat in mmlu:
#     mmlu_samples.extend(random.Random(0).sample(mmlu[cat], 10))

# with open('../data/mmlu_samples_140.pkl', 'wb') as f:
#     pickle.dump(mmlu_samples, f)

with open('../data/mmlu_samples_140.pkl', 'rb') as f:
    mmlu_samples = pickle.load(f)

In [ ]:
# res_mmlu_org = qa.mmlu_eval(mmlu_samples)
# sum([x['answer'] == x['r_org'] for x in res_mmlu_org])/len(res_mmlu_org)

# with open('../data/res_mmlu_org.pkl', 'wb') as f:
#     pickle.dump(res_mmlu_org, f)

Processing MMLU: 100%|██████████| 140/140 [00:10<00:00, 13.34it/s]


0.4642857142857143

In [5]:
with open('../data/res_mmlu_org.pkl', 'rb') as f:
    res_mmlu_org = pickle.load(f)

res_mmlu_one_rnd = qa.mmlu_eval(res_mmlu_org, "one_rnd")
sum([x['answer'] == x['r_org'] for x in res_mmlu_one_rnd])/len(res_mmlu_one_rnd)

with open('../data/res_mmlu_one_rnd.pkl', 'wb') as f:
    pickle.dump(res_mmlu_one_rnd, f)

qa


ValueError: This function should be called from the main script.

In [ ]:
import pickle



In [88]:
res_mmlu_org[0]

{'question_id': 468,
 'question': 'On October 25 RalphMuffetaccepted a draft for $620 that was dated October 10 and due 3 months after sight. It was discounted on November 16 at 4 (1/2)%. The bank charges (1/8)% collection fee. Find the proceeds of the draft.',
 'options': ['$625.00',
  '$608.90',
  '$613.79',
  '$600.25',
  '$620.00',
  '$615.50',
  '$5.43',
  '$618.47',
  '$610.33',
  '$0.78'],
 'answer': 'C',
 'answer_index': 2,
 'cot_content': '',
 'category': 'business',
 'src': 'stemez-Business',
 'r': 'C',
 'p_r': np.float64(0.866199836445556),
 'topk': [('C', np.float64(0.8662)),
  ('B', np.float64(0.1035)),
  ('H', np.float64(0.0204)),
  ('D', np.float64(0.0058)),
  ('I', np.float64(0.0028)),
  ('E', np.float64(0.0005)),
  ('A', np.float64(0.0004)),
  ('F', np.float64(0.0003)),
  ('G', np.float64(0.0001)),
  ('J', np.float64(0.0))]}